In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# ----------------------------
# 구성 변수
# ----------------------------
PDF_PATH = '../data/콘텐츠분쟁해결_사례.pdf'
EMBEDDING_MODEL_NAME = "solar-embedding-1-large"
LLM_MODEL_NAME = "solar-pro"
UPSTAGE_BASE_URL = "https://api.upstage.ai/v1"
VECTORSTORE_DIR = "faiss_index"

# ----------------------------
# 0단계: 문서 로드 및 1단계 분할
# ----------------------------
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

split_docs = text_splitter.split_documents(docs)
print(f"총 청크 수: {len(split_docs)}")

# ----------------------------
# 2단계: 임베딩 설정
# ----------------------------
embeddings = UpstageEmbeddings(model=EMBEDDING_MODEL_NAME)

# ----------------------------
# 3단계: 벡터스토어 생성
# ----------------------------
vectorstore = FAISS.from_documents(split_docs, embeddings)

# 선택: 로컬 저장
try:
    vectorstore.save_local(VECTORSTORE_DIR)
    print(f"벡터스토어 저장 완료: {VECTORSTORE_DIR}")
except Exception as e:
    print("벡터스토어 저장 실패:", e)

# 검색기 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# ----------------------------
# 4단계: LLM 설정
# ----------------------------
llm = ChatUpstage(
    model=LLM_MODEL_NAME,
    base_url=UPSTAGE_BASE_URL,
    temperature=0.2
)

# ----------------------------
# 5단계: 프롬프트 (과제용 간결 버전)
# ----------------------------
prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 사례를 근거로 답변하세요.
2. 관련 법령을 명시하세요.
3. 단계별 해결방안을 제시하세요.
4. 유사 사례를 참조하세요.
5. 없는 정보는 "확인할 수 없습니다"라고 하세요.
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# ----------------------------
# 6단계: QA 체인 생성
# ----------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# ----------------------------
# 7단계: 테스트 질문
# ----------------------------
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

# ----------------------------
# 8단계: QA 실행
# ----------------------------
for question in test_questions:
    print("\n" + "="*60)
    print(f"질문: {question}")

    # invoke 사용: 입력은 dict, 출력도 dict
    result_dict = qa_chain.invoke({"query": question})

    # 답변 추출
    answer_text = result_dict.get("result") or result_dict.get("output_text")
    print("답변:\n", answer_text)

    # 참조 문서 출력 (최대 3개)
    source_docs = result_dict.get("source_documents", [])
    if source_docs:
        print("\n참조 문서 (상위 3개):")
        for i, doc in enumerate(source_docs[:3], 1):
            preview = doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content
            meta = getattr(doc, "metadata", {})
            print(f"{i}. page={meta.get('page', meta.get('page_number', 'N/A'))}, preview='{preview}'")

